# Pro-social Dialog, Allen-AI


In [1]:
import pandas as pd
df_prosocial = pd.read_json('train.json', lines = True)
df_test = pd.read_json('test.json', lines = True)

In [2]:
df_prosocial.head()

,context,response,rots,safety_label,safety_annotations,safety_annotation_reasons,source,etc,dialogue_id,response_id,episode_done
0,I tried a bit of shadowboxing today. Got one g...,"This isn't a joke about black people, right? I...",[It's not right to think black people deserve ...,__needs_caution__,"[needs caution, needs caution, needs caution]",[It's rude to make jokes about the color of pe...,sbic,,0,0,False
1,"You're right, it isn't funny. Finding enjoymen...",I am glad that you agree. Joking about abusing...,[It's not right to think black people deserve ...,__casual__,"[casual, casual, casual]",[The speaker admits they spoke inappropriately...,sbic,,0,1,True
2,Did I tell you how I used to poke holes in con...,That's a pretty low down thing to do.,[It's immoral to poke holes in condoms.],__needs_intervention__,"[needs caution, needs intervention, needs inte...",[Speaker admits to committing a seriously wron...,socialchemistry,reddit/confessions/bsajnv,1,0,False
3,I wanted to prank others like that,You should not poke holes in condoms. You coul...,[It's harmful to tamper with other peoples con...,__needs_intervention__,"[needs caution, needs intervention, needs inte...","[Speaker explains why they did the act., They ...",socialchemistry,reddit/confessions/bsajnv,1,1,False
4,It's harmless. I doesn't hurt anyone. I just w...,You're not thinking. How do you conclude expos...,[It's normal for people to be alarmed by an un...,__needs_intervention__,"[needs caution, needs intervention, needs inte...",[Speaker doesn't see why what they did was wro...,socialchemistry,reddit/confessions/bsajnv,1,2,True


In [3]:
#check the length of cotext using a violon plot:
import seaborn as sb

df_prosocial['context_len']=[len(item) for item in df_prosocial['context']]
sb.violinplot(df_prosocial['context_len'])

In [8]:
# !pip install seaborn

In [4]:
# pip install transformers

In [9]:
#how many turns there are (eposode_done = True shows end of the turn)?
# len(df_prosocial[df_prosocial['episode_done']==False]),len(df_prosocial[df_prosocial['episode_done']==True])

In [4]:
#check how many classes there are:
set(df_prosocial[df_prosocial['episode_done']==True]['safety_label'])

{'__casual__',
 '__needs_caution__',
 '__needs_intervention__',
 '__possibly_needs_caution__',
 '__probably_needs_caution__'}

In [5]:
#get the number of sample in each class
from collections import Counter
Counter(df_prosocial.safety_label)

Counter({'__needs_caution__': 50493,
         '__casual__': 20690,
         '__needs_intervention__': 14605,
         '__possibly_needs_caution__': 16458,
         '__probably_needs_caution__': 17990})

# Classification using Transformers:

In [6]:
#get only the context, response, and the label columns
data = df_prosocial[['context','response', 'safety_label']]
data_test = df_test[['context', 'response', 'safety_label']]

### give context to each sample to improve classification


In [7]:
data['text'] = data['context'].astype(str)+' '+data['response']
df_test['text'] = df_test['context'].astype(str)+' '+df_test['response']

/tmp/ipykernel_1655832/956490412.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['context'].astype(str)+' '+data['response']


In [8]:
# revert to only include the context without responses
# data['text'] = data['context']
# data_test['text'] = data_test['context']

In [9]:
data = data.rename(columns={"safety_label":"labels"})
testset = data_test.rename(columns={"safety_label":"labels"})

### Subsample 10 % of the data -- (next, try on all data when we have resources)

In [15]:
df = data.sample(frac = .15, replace = False, random_state = 123456789)
len(df), len(data)

(18035, 120236)

In [16]:
# megering the cautious class to "bully" class, and convert the problem to a binary classification
# df['labels'] = df['labels'].replace(['__needs_caution__','__needs_intervention__','__probably_needs_caution__','__possibly_needs_caution__' ], 'bully')

In [17]:
from collections import Counter
Counter(df['labels'])

Counter({'__needs_caution__': 7580,
         '__casual__': 3123,
         '__needs_intervention__': 2251,
         '__probably_needs_caution__': 2643,
         '__possibly_needs_caution__': 2438})

# Prepare data and merge labels and data

In [18]:
import torch
import numpy as np

class prepareDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()} #why the encoding object gives the key directly but then val should be sent to torch.tensor ?
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __get_labels__(self, idx):
    return torch.tensor(self.labels[idx])

  def get_labels(self):
    return self.labels

  def __len__(self):
    return len(self.labels)

def encode_tags(labels, encodings):
  #make an empty list to return at the end of this function
  encoded_labels = []
  #reading documents labels and their offset from extracted labels and given encoding input
  for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
    #create an empty array of -100
    doc_end_labels = np.ones(len(doc_offset), dtype = int)* -100
    arr_offset = np.array(doc_offset)

    #can only get documents with len of max MAX_LEN; if the len is bigger, only get to the MAX_LEN of those docuemnts and the rest is -100
    #@Amin: but these are the labels. when does it take care of the input to not have len bigger than max_len?
    try:
      doc_end_labels[0:len(doc_labels)] = doc_labels
    except:
      doc_end_labels[0:MAX_LEN] = doc_labels

    #add the current encoded labels to the previouse list of encoded_labels
    encoded_labels.append(doc_end_labels.tolist())
  return encoded_labels

## Get the tokenizer from AutiTokenizer


In [20]:
from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
#testing distilbert instead of bert-base
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

### Subsample a balanced dataset based on labels..

In [22]:
df_labels = df['labels'].tolist()
df_input = df['text'].tolist()

## Downsampling the dataset -- first, try it without sampling!


In [23]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
df_in, df_labels = rus.fit_resample(df, df['labels'].tolist())
df_input = df_in['text'].tolist()
len(df_in)

11255

In [24]:
Counter(df_in['labels'])

Counter({'__casual__': 2251,
         '__needs_caution__': 2251,
         '__needs_intervention__': 2251,
         '__possibly_needs_caution__': 2251,
         '__probably_needs_caution__': 2251})

### split train to train and valiation sets:

In [25]:
from sklearn.model_selection import train_test_split
doc_set, doc_val, labels, labels_val = train_test_split(df_input, df_labels, test_size = .1, random_state = 123456789)

### Tokenize the train set and valiation set, the input data

In [26]:
MAX_LEN = 512
inputs = tokenizer(doc_set, truncation=True,  padding='max_length',max_length = MAX_LEN, return_offsets_mapping=True)
inputs_val =tokenizer(doc_val, truncation=True,  padding='max_length',max_length = MAX_LEN, return_offsets_mapping=True)

# Prepare dataset for the model!

In [ ]:
#this is for token classificaiton, and not sequence classification.
# labels = encode_tags(labels,inputs)

# Load the pre-trained BERT model

In [27]:
import torch
from transformers import BertModel
from transformers import BertTokenizer

#load the pre-trained BERT model
model_name = 'bert-base-uncased'
model = BertModel.from_pretrained(model_name)

MAX_LEN = 512
# num_class = 5
num_class = 2
# Add two layers of softmax -- > I think we added one?
class SequenceClassification(torch.nn.Module):
    def __init__(self):
        super(SequenceClassification, self).__init__()
        #self.fc1 = torch.nn.Linear(MAX_LEN*768, num_class) since we only need CLS, given it is a classification problem, we do not need to have MAX_LEN here :
        self.l1 = torch.nn.Linear(768, 512)
        self.activation1 = torch.nn.ReLU()
        self.dropout1 = torch.nn.Dropout(0.10)


        #layer 2:
        self.l2 = torch.nn.Linear(512, 256)
        self.activation2 = torch.nn.ReLU()
        self.dropout2 = torch.nn.Dropout(.1)


        # layer 3:
        self.fc = torch.nn.Linear(256, num_class)
        self.softmax = torch.nn.Softmax(dim=1)
        self.dropout3 = torch.nn.Dropout(.1)


    def forward(self, input_ids, attention_mask):
        outputs = model(input_ids, attention_mask,return_dict=False)
        pooled_output = outputs[1] # get the pooled output 1 (i.e., CLS token)
        pooled_output = self.dropout1(self.activation1(self.l1(pooled_output)))
        pooled_output = self.dropout2(self.activation2(self.l2(pooled_output)))
        pooled_output = self.dropout3(self.softmax(self.fc(pooled_output)))

        # x = self.softmax(self.fc1(torch.relu(pooled_output)))

        # pooled_output = self.dropout1(pooled_output)
        # output1 = self.l1(pooled_output)
        # input2 = self.dropout2(output1)
        # output2 = self.activation2(input2)
        # output2 = self.dropout2(output2)
        # x = self.softmax(output2)
        return pooled_output

# Use the SoftmaxLayers model
classification_model = SequenceClassification()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
print(classification_model)

SequenceClassification(
  (l1): Linear(in_features=768, out_features=512, bias=True)
  (activation1): ReLU()
  (dropout1): Dropout(p=0.1, inplace=False)
  (l2): Linear(in_features=512, out_features=256, bias=True)
  (activation2): ReLU()
  (dropout2): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=2, bias=True)
  (softmax): Softmax(dim=1)
  (dropout3): Dropout(p=0.1, inplace=False)
)


In [29]:
tag2id = {
      '__casual__' : 0,
      '__needs_caution__' : 1,
      '__needs_intervention__' :2,
      '__possibly_needs_caution__':3,
      '__probably_needs_caution__':4

  }

# ids for testing the model on a binary classification (i.e., casual and bully classes)
# tag2id = {
#       '__casual__' : 0,
#       'bully' : 1
#   }

id2tag = {v:k for k,v in tag2id.items()} #when and how do we want to use this id2tags?

In [30]:
id2tag

{0: '__casual__',
 1: '__needs_caution__',
 2: '__needs_intervention__',
 3: '__possibly_needs_caution__',
 4: '__probably_needs_caution__'}

In [31]:
#convert the string labels to ids
int_labels = [tag2id[tag] for tag in labels]
set(int_labels)

{0, 1, 2, 3, 4}

In [32]:
labeles_val_int = [tag2id[tag] for tag in labels_val]

In [33]:
#@ this prepare dataset only take care of the labels? do we use the input as returned by the tokenizer?
train_dataset = prepareDataset(inputs, int_labels)
val_dataset = prepareDataset(inputs_val, labeles_val_int)

In [34]:
from transformers import BertModel
from transformers import BertTokenizer

#load the pre-trained BERT model and fine-tune the data on a pre-trained model.
model_name = 'distilbert-base-cased'
model = BertModel.from_pretrained(model_name)


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at distilbert-base-cased were not used when initializing BertModel: ['distilbert.transformer.layer.0.attention.out_lin.bias', 'distilbert.transformer.layer.0.attention.k_lin.bias', 'distilbert.transformer.layer.4.ffn.lin1.weight', 'distilbert.transformer.layer.0.ffn.lin2.bias', 'distilbert.transformer.layer.3.ffn.lin1.bias', 'distilbert.transformer.layer.1.attention.q_lin.weight', 'distilbert.transformer.layer.2.attention.k_lin.bias', 'distilbert.transformer.layer.3.output_layer_norm.weight', 'distilbert.transformer.layer.2.attention.q_lin.weight', 'distilbert.transformer.layer.5.attention.v_lin.bias', 'distilbert.transformer.layer.2.sa_layer_norm.weight', 'distilbert.transformer.layer.1.attention.k_lin.weight', 'distilbert.transformer.layer.3.attention.k_lin.weight', 'distilbert.transformer.layer

In [36]:
#using hugging face model instead of ours:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

classification_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-cased", num_labels=5, id2label=id2tag, label2id=tag2id, from_tf=False
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifier.w

In [49]:
print(classification_model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [41]:
# pip install torchsampler

In [37]:
from torchsampler import ImbalancedDatasetSampler

/opt/tljh/user/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/tljh/user/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIlEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  warn(f"Failed to load image Python extension: {e}")


In [38]:
from collections import Counter
Counter(int_labels)

Counter({1: 2045, 3: 2012, 2: 2021, 4: 2027, 0: 2024})

In [56]:
#fine-tuning the model
import torch
import logging
from torch.utils.data import DataLoader
from transformers import AdamW

#use CPU if the GPU is not avaiable
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
if not torch.cuda.is_available():
  logging.warning('GPY is not available. The modeing is being trained on CPU!')

#preapre the model for training
# model.config.id2label = id2tag
model.to(device)
model.train()
#DO WE ALSO PUT THE MODEL FOR FINED TUNNING ON THE TRAIN PHASE AND THE BERT MODEL ITESELF?
classification_model.to(device)
classification_model.train()

BSIZE = 8#batch size
#preparing the data for PyTorch (does it mean to only convert it to batches?)
# train_loader = DataLoader(train_dataset, batch_size=BSIZE, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=BSIZE, sampler = ImbalancedDatasetSampler(train_dataset), shuffle=False)

val_loader = DataLoader(val_dataset, batch_size=BSIZE, sampler = ImbalancedDatasetSampler(val_dataset), shuffle=False)
#set the optimizer
optim = AdamW(model.parameters(), lr=5e-5)
#set the criterion to compute error
criterion = torch.nn.CrossEntropyLoss()
#print the performance metrics over the training phase every iteration_reset=100
iteration_reset = 100

'''max of epochs. Change this responsibily. Higher values for epochs can cause overfitting.
'''
MAX_EPOCH = 2

#main loop for epochs
for epoch in range(MAX_EPOCH):
    train_loss = 0.0
    val_loss = 0.0
    loss_sum = 0.0

    #batch training loop
    for i,batch in enumerate(train_loader):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = classification_model(input_ids, attention_mask=attention_mask)[0]
        # loss = outputs[0]
        # loss = criterion(outputs.view(-1, tokenizer.vocab_size),labels.view(-1))
        loss  = criterion(outputs,labels)
        loss_sum += loss.to('cpu').detach().numpy()
        train_loss += loss.to('cpu').detach().numpy()
        loss.backward()
        optim.step()

        #printing loss
        if i% iteration_reset == 0 and i>0:
            print('Epoch {0}/{1}, batch {2}/{3}, Loss: {4}'.format(epoch,MAX_EPOCH-1,i,len(train_loader),
                                                                   loss_sum/(iteration_reset*BSIZE)))
            loss_sum = 0
    #compute model performance during training, and after each epoch:
    model.eval() #put the model to the val phase; in other words, turn off the training phase for this model
    classification_model.eval()
    with torch.no_grad():
      for i, batch in enumerate(val_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = classification_model(input_ids, attention_mask = attention_mask)
        loss = criterion(outputs[0], labels)
        val_loss +=loss.to('cpu').detach().numpy()
    print('Epoch {0} finished with train loss of {1} and validation loss of {2}'.format(epoch, train_loss/(BSIZE*len(train_loader)), val_loss/(BSIZE*len(val_loader))))
    model.train() #put the model to the train phase..
    classification_model.train()

model.eval()
classification_model.eval() #finally, we set off the training and put the model into validation for prediction use only

/opt/tljh/user/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0/1, batch 100/8216, Loss: 0.20358751371502876
Epoch 0/1, batch 200/8216, Loss: 0.20169284299016
Epoch 0/1, batch 300/8216, Loss: 0.20144096240401269
Epoch 0/1, batch 400/8216, Loss: 0.20159250393509864
Epoch 0/1, batch 500/8216, Loss: 0.20176831349730492
Epoch 0/1, batch 600/8216, Loss: 0.20141353502869605
Epoch 0/1, batch 700/8216, Loss: 0.20136508628726005
Epoch 0/1, batch 800/8216, Loss: 0.20143188580870627
Epoch 0/1, batch 900/8216, Loss: 0.20108332514762878
Epoch 0/1, batch 1000/8216, Loss: 0.20207090839743613
Epoch 0/1, batch 1100/8216, Loss: 0.20136445447802542
Epoch 0/1, batch 1200/8216, Loss: 0.20190103828907013
Epoch 0/1, batch 1300/8216, Loss: 0.20148227602243424
Epoch 0/1, batch 1400/8216, Loss: 0.20121277183294295
Epoch 0/1, batch 1500/8216, Loss: 0.20132773533463477
Epoch 0/1, batch 1600/8216, Loss: 0.2013779279589653
Epoch 0/1, batch 1700/8216, Loss: 0.2015802749991417
Epoch 0/1, batch 1800/8216, Loss: 0.2011352077126503
Epoch 0/1, batch 1900/8216, Loss: 0.2017877

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [58]:
classification_model.save_pretrained('classification_prosocial_finetuned_distilbert-base-cased',from_pt=True)

In [41]:
#load the model.
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

classification_model = AutoModelForSequenceClassification.from_pretrained(
    "classification_prosocial_finetuned_distilbert-base-cased", from_tf=False
)

In [59]:
print('Epoch {0} finished with train loss of {1} and validation loss of {2}'.format(epoch, train_loss/(BSIZE*len(train_loader)), val_loss/(BSIZE*len(val_loader))))

Epoch 1 finished with train loss of 0.2015264265406097 and validation loss of 0.20145091971000992


In [79]:
outputs[0]

tensor([[-0.0535,  0.1878,  0.0765,  0.0595,  0.0479],
        [-0.0461,  0.1807,  0.0341,  0.0656,  0.0566],
        [-0.0408,  0.1574,  0.0410,  0.0662,  0.0637],
        [-0.0720,  0.2017,  0.0259,  0.0758,  0.0496],
        [-0.0481,  0.1889,  0.0401,  0.0750,  0.0644],
        [-0.0692,  0.1807,  0.0185,  0.0900,  0.0663],
        [-0.0436,  0.1791,  0.0269,  0.0672,  0.0676]], device='cuda:0')

In [78]:
labels

tensor([4, 0, 1, 3, 0, 4, 3], device='cuda:0')

In [77]:
torch.argmax(outputs[0], dim=1)

tensor([1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [ ]:
model.eval() 
with torch.no_grad():
  for i, batch in enumerate(val_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    outputs = model(input_ids, attention_mask = attention_mask, labels = labels)
    loss = outputs.logits
    val_loss +=loss.to('cpu').detach().numpy()
print('Epoch {0} finished with train loss of {1} and validation loss of {}'.format(epoch, train_loss/(BSIZE*len(train_loader))), val_loss/(BSIZE*len(val_loader)))
model.train() #put the model to the train phase..

### Compute performance after tranining:

In [89]:
testset['text'] =testset['context'].astype(str)+' '+testset['response'].astype(str)

In [45]:
# MAX_LEN = 512
doc_test_set = testset['text'].tolist()
labels_test_set = testset['labels'].tolist()
int_labels_test_set = [tag2id[tag] for tag in labels_test_set]

#### prepare the test set :

In [43]:
# TEST BLOCK
inputs_ids_test = tokenizer.encode_plus(doc_test_set[0])
inputs_ids_test

In [46]:
torch.argmax(classification_model(inputs_ids_test['input_ids'].to(device), attention_mask = inputs_ids_test['attention_mask'].to(device))[0])

In [98]:
# TEST BLOCK
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

classification_model(inputs_ids_test['input_ids'].to(device), attention_mask = inputs_ids_test['attention_mask'].to(device))
# classification_model(inputs_ids_test['input_ids'].to(device), attention_mask = inputs_ids_test['attention_mask'].to(device)).config
torch.argmax(classification_model(inputs_ids_test['input_ids'].to(device), attention_mask = inputs_ids_test['attention_mask'].to(device))[0] ,dim=1)

tensor([1], device='cuda:0')

In [100]:
classification_model(inputs_ids_test['input_ids'].to(device), attention_mask = inputs_ids_test['attention_mask'].to(device))[0]

tensor([[-0.0456,  0.1853,  0.0559,  0.0605,  0.0449]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

### Run the classification model on the test set. 

In [63]:
#1. tokenize the test set, run the classification, get outputs, and append the results to later compute confusion matrix
test_res = []
for item in doc_test_set:
  inputs_ids_test = tokenizer.encode_plus(item, padding=True, truncation=True, max_length=MAX_LEN, return_tensors='pt')
  tests_output = torch.argmax(classification_model(inputs_ids_test['input_ids'].to(device), attention_mask = inputs_ids_test['attention_mask'].to(device))[0] ,dim=1).item()
  test_res.append(tests_output)
len(test_res)

25029

# Run the performance metric on the test set

In [47]:
from sklearn.metrics import confusion_matrix
confusion_matrix(int_labels_test_set, test_res)